In [5]:
import argparse
import io

from IPython.core.display import display, HTML
import IPython.display as ipd


from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

from pydub import AudioSegment

import os
from scipy.io import wavfile
from scipy.fftpack import fft
import librosa
import re


import alignment_new

from pymongo import MongoClient

import json

 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/nikitos/HeedBook-59df779e8e5a.json'

client = speech.SpeechClient()


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yaml
import time
import glob
import sys
from datetime import datetime, timedelta
import pymorphy2
from collections import Counter

from tqdm import tqdm_notebook


In [7]:
def play_mp3(audio):
    audio.export('/Users/nikitos/export/current.wav', format='wav')
    audio, rt = librosa.load('/Users/nikitos/export/current.wav')
    display(ipd.Audio(audio, rate=rt))

In [8]:
def export_to_raw(audio):
    audio.export('/Users/nikitos/export/to_raw.wav', 'wav')
    os.system('sox -v 0.99 /Users/nikitos/export/to_raw.wav -b 16 ./input.raw channels 1 rate 16000')
    return './input.raw'

In [9]:
def clear_text(text):        
        text = re.sub(r'[^\w-]+', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\s+$', '', text)
        text = text.lower()
        text = text.replace('ё', 'е')
        return text

In [10]:
def transcribe_file(speech_file):
    
    out = []
    
    client = speech.SpeechClient()

    with io.open(speech_file, 'rb') as audio_file:
        content = audio_file.read()

    audio = types.RecognitionAudio(content=content)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code='ru-RU',
        enable_word_time_offsets=True)
    
    response = client.recognize(config, audio)
    transcript = ''
    for result in response.results:
        alternative = result.alternatives[0]
        transcript = alternative.transcript
        #print('Transcript: {}'.format(alternative.transcript))

        for word_info in alternative.words:
            curr_word = {}
            word = word_info.word
            curr_word['Word'] = word
            start_time = word_info.start_time
            end_time = word_info.end_time
            curr_word['Begin'] = start_time.seconds + start_time.nanos * 1e-9
            curr_word['End'] = end_time.seconds + end_time.nanos * 1e-9
            out.append(curr_word)
            answ = 'Word: {}, start_time: {}, end_time: {}'.format(
                word,
                start_time.seconds + start_time.nanos * 1e-9,
                end_time.seconds + end_time.nanos * 1e-9)
            
            #print(answ)
    return response, out, transcript

In [11]:
def search_text_frame(transcript, ORIGINAL_TEXT, text_list):
    phrase_list = transcript.split()
    size_phrase = min(10, len(phrase_list))

    N = size_phrase + 6
    PERCENT = 0.8

    TABOO_PHRASE = ' '.join(phrase_list[:size_phrase])

    pack, length = word_search(N, PERCENT, TABOO_PHRASE, ORIGINAL_TEXT)
    for_alignment = text_list[pack[0][0][1]:pack[0][0][1] + len(phrase_list) + 2]
    return for_alignment

In [12]:
def making_data_list(align1, align2, curr_list):
    data_list = []
    skip_index = 0
    for i in range(len(align1)):
        real_word = align1[i]
        recogn_word = align2[i]
        if ('#' in real_word) and ('#' in recogn_word):
            skip_index += 1
            continue
        elif ('#' in real_word):
            continue
        elif ('#' in recogn_word):
            skip_index += 1
            continue
        #print(real_word, recogn_word)
        if real_word == recogn_word:
            sample = {}
            sample['text_word'] = real_word
            sample['regonized_word'] = recogn_word
            sample['start'] = curr_list[i - skip_index]['Begin']
            sample['end'] = curr_list[i - skip_index]['End']
            sample['confidance'] = 1
            sample['file'] = path
            data_list.append(sample)
        elif real_word != recogn_word:
            sample = {}
            sample['text_word'] = real_word
            sample['regonized_word'] = recogn_word
            sample['start'] = curr_list[i - skip_index]['Begin']
            sample['end'] = curr_list[i - skip_index]['End']
            sample['confidance'] = 0
            sample['file'] = path
            data_list.append(sample)
    return data_list

In [50]:
def word_search(N, PERCENT, TABOO_PHRASE, ORIGINAL_TEXT):

    morph = pymorphy2.MorphAnalyzer()
    sns.set() 
    
    def normalize_word(word):
        # todo: hardcode
        for i in range(10):
            prev_word = word
            word = morph.parse(word)[0].normal_form
            if word == prev_word:
                return word
        return word

    # text goes to normal form
    def normalize_text(text):
        return ' '.join([normalize_word(word) for word in text.split(' ')])

    # generation list of synonyms
    def find_taboo(words, taboo_synonyms_df):
#         words: taboo words to find
#         taboo_synonyms_df: found synonyms in text. pd.DataFrame(columns=['word']) with index, corresponding to list of <words> from the text

#         return: a list of values of found words: [[<word>, <index>, <synonym or original word found in text>], ...]
#         example: [[['a', 0, 'a']], [['b', 2, 'b']], [['c', 3, 'd']]]
        words = list(words)

        res_values = []
        for i in range(len(words)):
            print(i, taboo_synonyms_df, words)
            res_values.append([words[i]] + taboo_synonyms_df.iloc[i].tolist())
        return res_values
    
    
    text = normalize_text(ORIGINAL_TEXT)
    words = text.split(' ')
    words_df = pd.DataFrame(words, columns=['word']).reset_index()
    taboo_phrase = normalize_text(TABOO_PHRASE)
    taboo_words = taboo_phrase.split(' ')


    taboo_synonyms = taboo_words
        
    taboo_index = [i for i, word in enumerate(words) if word in taboo_synonyms]
    taboo_words_df = words_df.iloc[taboo_index, :]
   
    cond1 = taboo_words_df['index'] - taboo_words_df['index'].shift(1).fillna(0) < N
    cond2 = taboo_words_df['index'].shift(-1).fillna(0) - taboo_words_df['index'] < N
    taboo_words_df = taboo_words_df[cond1 | cond2]
    res = []
    for i in range(taboo_words_df.shape[0]):
        index_low = taboo_words_df.index[i]
        index_high = index_low + N
        indices_in = [i for i in taboo_words_df.index if i > index_low - 1 and i < index_high + 1]
        taboo_words_in = taboo_words_df.loc[indices_in]
        pack = find_taboo(taboo_words, taboo_words_in)
        pack = list(sorted(pack, key=lambda x: x[1]))
        if len(pack) > round(PERCENT * len(taboo_words)) - 1:
            res.append(pack)
            break

    
    final_pack = []

    original_words = ORIGINAL_TEXT.split(' ')
    beg_last = 0
    end_last = 0
    j = 0
    for pack in res:
        pack_synonyms = list(set([x[2] for x in pack]))
        beg, end = pack[0][1], pack[-1][1] + 1
        real_end = beg + N
        if end - beg > N:
            beg_last = beg
            end_last = end
            continue

        if end == end_last:
            beg_last = beg
            continue

        tmp = [0] * len(taboo_words)
        for i in range(beg, real_end):
            if words[i] in taboo_words:
                index = taboo_words.index(words[i])
                tmp[index] = 0 

        s = 0
        for i in range(len(tmp)):
             s = s + tmp[i]

        if s!= 0:
            continue

        final_pack.append(pack)
        break
        j = j + 1

        end_last = end
        beg_last = beg
        
    return final_pack

In [51]:
def data_from_audiobooks(audio_path, text_path):
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/nikitos/HeedBook-59df779e8e5a.json'
    client = speech.SpeechClient()
    audio = AudioSegment.from_mp3(audio_path)
    
  
    ORIGINAL_TEXT = open(text_path, encoding='windows-1251').read()
    ORIGINAL_TEXT = clear_text(ORIGINAL_TEXT)
    
    text_list = ORIGINAL_TEXT.split()
    
    total_mls = len(audio)
    step = 30000
    number = 1
    data = []
    while step * number < total_mls:
        start_fr = step * number
        number += 1
        end_fr = step * number
        x, curr_list, transcript = transcribe_file(export_to_raw(audio[start_fr:end_fr]))
        transcript = clear_text(transcript)
        phrase_list = transcript.split()
        for_alignment = search_text_frame(transcript, ORIGINAL_TEXT, text_list)
        
        align1, align2 = alignment_new.needle(for_alignment, phrase_list)
        
        list_for_frame = making_data_list(align1, align2, curr_list)
        data += list_for_frame
        
    start_fr = step * number
    end_fr = total_mls - 1
    x, curr_list, transcript = transcribe_file(export_to_raw(audio[start_fr:end_fr]))
    transcript = clear_text(transcript)
    phrase_list = transcript.split()
    for_alignment = search_text_frame(transcript, ORIGINAL_TEXT, text_list)
        
    align1, align2 = alignment_new.needle(for_alignment, phrase_list)
        
    list_for_frame = making_data_list(align1, align2, curr_list)
    data += list_for_frame
    
    
    return data

In [ ]:
OR

In [52]:
data = data_from_audiobooks('./generation_p/book/0002.mp3', './generation_pi.txt')

0     index word
18     18    в
24     24   на
28     28    в ['уравновесить', 'светлый', 'начать', 'тёмный', 'несмотря', 'на', 'этот', 'блестящий', 'разработка', 'в']
1     index word
18     18    в
24     24   на
28     28    в ['уравновесить', 'светлый', 'начать', 'тёмный', 'несмотря', 'на', 'этот', 'блестящий', 'разработка', 'в']
2     index word
18     18    в
24     24   на
28     28    в ['уравновесить', 'светлый', 'начать', 'тёмный', 'несмотря', 'на', 'этот', 'блестящий', 'разработка', 'в']
3     index word
18     18    в
24     24   на
28     28    в ['уравновесить', 'светлый', 'начать', 'тёмный', 'несмотря', 'на', 'этот', 'блестящий', 'разработка', 'в']


IndexError: single positional indexer is out-of-bounds